In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Incremental Training with Different Learning Rates and Layer-Freezing

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container. 

In this example, we fine-tune a model by setting different learning rates to different layers and freezing embedding tables. Incremental training of a model is a common practice allows models to continuously learn and extend the existing model's knowledge by adjusting model parameters that has been learned previously using new examples. Another scenario to do incremental training is to resume a training job that was stopped. Here, we first showcase how to incrementally train the same model architecture with different hyperparameter settings (adjusting the learning rates) and using different datasets. Then, in a new scenario, we showcase how one can freeze certain layers of the model such as pretrained embedding layers and perform training.

**Learning objectives**
- Training a model with multiple learning rates
- Fine-tune a model by freezing embedding tables

In [2]:
import os

import tensorflow as tf
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

from merlin.datasets.synthetic import generate_data
import merlin.models.tf as mm
from merlin.schema import Schema, Tags
from merlin.io.dataset import Dataset

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-10-20 20:26:21.720796: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-20 20:26:24.477686: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:222] Using CUDA malloc Async allocator for GPU: 0
2022-10-20 20:26:24.477886: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8080 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:0a:00.0, compute capability: 7.0
/usr/local/lib/pyt

## Building a Two-Tower Model with Merlin Models

We choose Two-Tower model architecture for this example. To learn more about a Two-Tower model you can visit this [notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb)

### Generate Synthetic Dataset

Let's create three datasets. To generate the synthetic dataset for our example, we can use `generate_data()` function. We can assume that each dataset here was collected at a different day. Therefore, below we show how we can do incremental training with Merlin Models. We generate three datasets synthetically and we name them as `day_1, day_2, day_3` for this example.

In [3]:
NUM_ROWS = 10000
day_1, day_2, day_3 = generate_data("e-commerce-large", int(NUM_ROWS), set_sizes=(0.33, 0.33, 0.34))
schema = day_1.schema.without(['click', 'conversion'])
day_1.schema = schema
day_2.schema = schema
day_3.schema = schema

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [4]:
day_1.schema

,name,tags,dtype,is_list,is_ragged,properties.domain.min,properties.domain.max
0,user_categories,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,6086
1,user_shops,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,116741
2,user_brands,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,58015
3,user_intentions,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,33786
4,user_profile,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,98
5,user_group,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,14
6,user_gender,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,3
7,user_age,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,8
8,user_consumption_1,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,4
9,user_consumption_2,"(Tags.USER, Tags.CATEGORICAL)",int64,False,False,0,4


### Iteration 1: Using Different Learning Rates
At first, we train the model on the first day's data and evaluate it on the second day's data.

Define the embeddings for features in the item and query towers using `Embedddings` class. By setting `infer_embedding_sizes` to True, we can automatically define the embedding dimension from the feature cardinality in the schema.

In [5]:
item_embeddings = mm.Embeddings(schema.select_by_tag(Tags.ITEM), infer_embedding_sizes=True)
query_embeddings = mm.Embeddings(schema.select_by_tag(Tags.USER), infer_embedding_sizes=True)

Build the Two-Tower model

In [6]:
model = mm.TwoTowerModel(schema, 
                         query_tower=mm.InputBlockV2(schema.select_by_tag(Tags.USER), categorical=query_embeddings).connect(mm.MLPBlock([128, 64])), 
                         item_tower=mm.InputBlockV2(schema.select_by_tag(Tags.ITEM), categorical=item_embeddings).connect(mm.MLPBlock([128, 64])),
)

#### Training the model with first day's data

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))
model.fit(day_1, batch_size=1024, epochs=1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The sampler InBatchSampler returned no samples for this batch.


4/4 [==============================] - 17s 113ms/step - loss: 6.8098 - recall_at_10: 0.0211 - mrr_at_10: 0.0186 - ndcg_at_10: 0.0192 - map_at_10: 0.0186 - precision_at_10: 0.0021 - regularization_loss: 0.0000e+00


**Evaluate the data using day_2 dataset**

Training model on a certain period of time, and then evaluating on a dataset which is closer to the test set time period make sense and is a common practice. However, please note that in this example the data is randomly seperated, thus we dont really expect a temporal sequence in day_1, day_2, and day_3. Therefore the evaluation metrics might not "make sense", since this is a hypotetical example to showcase the functionality.

In [8]:
eval_metrics = model.evaluate(day_2, batch_size=1024, return_dict=True)
eval_metrics

The sampler InBatchSampler returned no samples for this batch.


4/4 [==============================] - 2s 12ms/step - loss: 6.8096 - recall_at_10: 0.0245 - mrr_at_10: 0.0217 - ndcg_at_10: 0.0223 - map_at_10: 0.0217 - precision_at_10: 0.0024 - regularization_loss: 0.0000e+00


{'loss': 6.809643268585205,
 'recall_at_10': 0.025151515379548073,
 'mrr_at_10': 0.021524891257286072,
 'ndcg_at_10': 0.022327682003378868,
 'map_at_10': 0.021524891257286072,
 'precision_at_10': 0.0025151518639177084,
 'regularization_loss': 0.0}

#### Training the model with second day's data 

Now we continue to train the model on the second day's data but using different strategies. We can use different learning rate for different layers of the model, i.e. a smaller learning rate for embedding tables while a bigger learning rate for two towers. If we want small updates to the weights of embedding tables, we can set the small learning rate value. Here we choose `0.001` as the learning rate for embedding tables.

In [9]:
optimizer = mm.MultiOptimizer(
            default_optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
            optimizers_and_blocks=[mm.OptimizerBlocks(tf.keras.optimizers.Adam(learning_rate=0.001),
                                                      [item_embeddings, query_embeddings])]
)                                      

In [10]:
model.compile(optimizer=optimizer)
model.fit(day_2, batch_size=1024, epochs=1)

4/4 [==============================] - 8s 315ms/step - loss: 6.8098 - recall_at_10: 0.0250 - mrr_at_10: 0.0211 - ndcg_at_10: 0.0219 - map_at_10: 0.0211 - precision_at_10: 0.0025 - regularization_loss: 0.0000e+00


**Evaluate on the third day's data**

In [11]:
eval_metrics = model.evaluate(day_3, batch_size=1024,  return_dict=True)

4/4 [==============================] - 2s 12ms/step - loss: 6.8036 - recall_at_10: 0.0290 - mrr_at_10: 0.0246 - ndcg_at_10: 0.0256 - map_at_10: 0.0246 - precision_at_10: 0.0029 - regularization_loss: 0.0000e+00


In [12]:
eval_metrics

{'loss': 6.803572654724121,
 'recall_at_10': 0.028823530301451683,
 'mrr_at_10': 0.022923553362488747,
 'ndcg_at_10': 0.024233652278780937,
 'map_at_10': 0.022923553362488747,
 'precision_at_10': 0.002882353262975812,
 'regularization_loss': 0.0}

### Iteration 2: Training with Freezing Layers

"Let's consider a new situation. Suppose we have trained the model on all previous data and achieved a good performance. Now there is incoming new data, but we do not want to change the pretrained embedding tables and only want to train the top MLP layers. We can use `model.freeze_blocks()`. When we call `freeze_blocks`, what do we actually do? Each layer maintains a variable called `trainable`. When a layer is created, this variable is set. The default value is `True`, which means all the weights in this layer can be updated. If you change `trainable` into `False`, the weights would not be changed anymore, unless its `trainable` variable becomes `True` again. So when `freeze_blocks` is called, the `trainable` of the layer is set to `False`.

In [13]:
model.freeze_blocks([item_embeddings, query_embeddings])

# recompile your model after making any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)) 

In [14]:
model.fit(day_1, batch_size=1024, epochs=1)
model.summary(expand_nested=True, show_trainable=True, line_length=80)  

4/4 [==============================] - 5s 22ms/step - loss: 6.7985 - recall_at_10: 0.0319 - mrr_at_10: 0.0265 - ndcg_at_10: 0.0277 - map_at_10: 0.0265 - precision_at_10: 0.0032 - regularization_loss: 0.0000e+00
Model: "retrieval_model"
___________________________________________________________________________________________
 Layer (type)                       Output Shape                    Param #     Trainable  
 two_tower_block (TwoTowerBlock)    multiple                        340774200   Y          
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| tower_block (TowerBlock)         multiple                        21941120    Y          |
||¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯||
|| sequential_block_3 (SequentialBloc  multiple                   21941120    Y          ||
|| k)                                                                                    ||
|||¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯

When we call `freeze_blocks` on some layers, all these layers and their children layers become non-trainable. For example, if a `ParallelBlock` is frozen, the children blocks inside this `ParallelBlock` are also frozen. As shown in below summary result, we freeze the `user_embeddings`, and it is a `ParallelBlock`, all the children layers are frozen as well.

```
|||||| embeddings (ParallelBlock)  multiple                   21902016    N          ||||||
|||||||¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|||||||
||||||| user_categories (EmbeddingTable)  multiple           146088      N          |||||||
|||||||                                                                             |||||||
||||||| user_shops (EmbeddingTable)  multiple                4669680     N          |||||||
|||||||                                                                             |||||||
||||||| user_brands (EmbeddingTable)  multiple               1856512     N          |||||||
|||||||                                                                             |||||||
||||||| user_intentions (EmbeddingTable)  multiple           1081184     N          |||||||
```

## Summary

In this example notebook we learned how to use different learning rates for different layers in our model architecture, and how to freeze embedding layers, so that we do not update their parameters during training.